In [3]:
import fitz
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from PIL import Image 
import io
import boto3
import numpy as np

In [65]:
from PIL import Image
import pytesseract
import argparse
import cv2
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import csv
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet 

In [66]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [73]:
%%time
data = {"Vat Number":'',
       "Total":'',
       "Date":''}
img  = cv2.imread('Makro Invoices/Makro Invoices/59839_A28647451 DEFAULT C9627 TYPE_20220520_6_FacturaCompra_page1.png', cv2.COLOR_RGB2GRAY)
text=(pytesseract.image_to_string(img))
text_list = text.split("\n")
print(type(text_list))
for idx, a in enumerate(text_list):
    if(a.strip() != ""):
        if("NIF:" in a or  "N.I.F.:" in a):
                print("Case Vat Number")
                b = a.split(" ")
                if len(b[-1])>10:
                    data["Vat Number"] = b[-1]
                    print(b[-1])
        if ("Tarjeta" in a or "Credito" in a or  "pagina" in a):
            print("Case Total")
            b = a.split(" ")

            data["Total"] = b[-1]
            print(b[-1])
        if ("vent:" in a or "venta:" in a or "Fecha:" in a):
            print("Date")
            b = a.split(" ")
            if len(b[-1])>7:
                data["Date"] = b[-1]
                print(b[-1])
            else:
                data["Date"] = b[-2]
                print(b[-2])


<class 'list'>
Date
20/05/2022
Case Vat Number
A-28/647451
Date
20/05/202
Case Total
334,57
Wall time: 9.16 s


In [74]:
data

{'Vat Number': 'A-28/647451', 'Total': '334,57', 'Date': '20/05/202'}

# Code for Converting PDF to IMG

In [4]:

def convert_pdf2img(input_file: str,pages=None):
    """Converts pdf to image and generates a file by page"""
    # Open the document
    pdfIn = fitz.open(input_file)
    output_files = []
    # Iterate throughout the pages
    for pg in range(pdfIn.pageCount):
        if str(pages) != str(None):
            if str(pg) not in str(pages):
                continue
        # Select a page
        page = pdfIn[pg]
        rotate = int(0)
        # PDF Page is converted into a whole picture 1056*816 and then for each picture a screenshot is taken.
        # zoom = 1.33333333 -----> Image size = 1056*816
        # zoom = 2 ---> 2 * Default Resolution (text is clear, image text is hard to read)    = filesize small / Image size = 1584*1224
        # zoom = 4 ---> 4 * Default Resolution (text is clear, image text is barely readable) = filesize large
        # zoom = 8 ---> 8 * Default Resolution (text is clear, image text is readable) = filesize large
        zoom_x = 2
        zoom_y = 2
        # The zoom factor is equal to 2 in order to make text clear
        # Pre-rotate is to rotate if needed.
        mat = fitz.Matrix(zoom_x, zoom_y).preRotate(rotate)
        pix = page.getPixmap(matrix=mat, alpha=False)
        path = 'Makro Invoices/Makro Invoices/'
        output_file = f"{os.path.splitext(os.path.basename(input_file))[0]}_page{pg+1}.png"
        pix.writePNG(output_file)
        output_files.append(output_file)
    pdfIn.close()
    summary = {
        "File": input_file, "Pages": str(pages), "Output File(s)": str(output_files)
    }
    # Printing Summary
    print("## Summary ########################################################")
    print("\n".join("{}:{}".format(i, j) for i, j in summary.items()))
    print("###################################################################")
    return output_files

In [6]:
convert_pdf2img('Makro Invoices/Makro Invoices/59537_A28647451 TIPO 91FF8_20220519_6_FacturaCompra.pdf')

Deprecation: 'preRotate' removed from class 'Matrix' after v1.19 - use 'prerotate'.
Deprecation: 'getPixmap' removed from class 'Page' after v1.19 - use 'get_pixmap'.


## Summary ########################################################
File:Makro Invoices/Makro Invoices/59537_A28647451 TIPO 91FF8_20220519_6_FacturaCompra.pdf
Pages:None
Output File(s):['59537_A28647451 TIPO 91FF8_20220519_6_FacturaCompra_page1.png']
###################################################################


Deprecation: 'writePNG' removed from class 'Pixmap' after v1.19 - use 'save'.


['59537_A28647451 TIPO 91FF8_20220519_6_FacturaCompra_page1.png']

# Code for Cropping Image

In [5]:

# Opens a image in RGB mode 
im = Image.open(r"Makro Invoices/Makro Invoices/59537_A28647451 TIPO 91FF8_20220519_6_FacturaCompra_page1.png") 
# Size of the image in pixels (size of orginal image) 
# (This is not mandatory) 
width, height = im.size 
print(im.size)
# Setting the points for cropped image 
left = 5
top = 250
right = 1678
bottom = 1600
# Cropped image of above dimension 
# (It will not change orginal image) 
im1 = im.crop((left, top, right, bottom)) 
# Shows the image in image viewer 
im1.save("Makro Invoices/Makro Invoices/pp.jpg")

(1191, 1684)


# Code for Extracting Table 

In [75]:
im = Image.open('Makro Invoices/Makro Invoices/59839_A28647451 DEFAULT C9627 TYPE_20220520_6_FacturaCompra_page1.png')

In [76]:
buffered = io.BytesIO()
im.save(buffered, format='PNG')

In [77]:
client = boto3.client('textract')
response = client.analyze_document(
    Document={'Bytes': buffered.getvalue()},
    FeatureTypes=['TABLES']
)

In [78]:
def map_blocks(blocks, block_type):
    return {
        block['Id']: block
        for block in blocks
        if block['BlockType'] == block_type
    }

blocks = response['Blocks']
tables = map_blocks(blocks, 'TABLE')
cells = map_blocks(blocks, 'CELL')
words = map_blocks(blocks, 'WORD')
selections = map_blocks(blocks, 'SELECTION_ELEMENT')

In [79]:
def get_children_ids(block):
    for rels in block.get('Relationships', []):
        if rels['Type'] == 'CHILD':
            yield from rels['Ids']

In [80]:
import pandas as pd

dataframes = []

for table in tables.values():

    # Determine all the cells that belong to this table
    table_cells = [cells[cell_id] for cell_id in get_children_ids(table)]

    # Determine the table's number of rows and columns
    n_rows = max(cell['RowIndex'] for cell in table_cells)
    n_cols = max(cell['ColumnIndex'] for cell in table_cells)
    content = [[None for _ in range(n_cols)] for _ in range(n_rows)]

    # Fill in each cell
    for cell in table_cells:
        cell_contents = [
            words[child_id]['Text']
            if child_id in words
            else selections[child_id]['SelectionStatus']
            for child_id in get_children_ids(cell)
        ]
        i = cell['RowIndex'] - 1
        j = cell['ColumnIndex'] - 1
        content[i][j] = ' '.join(cell_contents)

    # We assume that the first row corresponds to the column names
    dataframe = pd.DataFrame(content[1:], columns=content[0])
    dataframes.append(dataframe)

In [81]:
if "MM Num. artículo" in dataframes[2]:
    print("Hi")
    print(len(dataframes))

Hi
4


In [82]:

if "MM Num. articulo" in dataframes[1] or "MM Num. artículo" in dataframes[1]:
    data = pd.DataFrame(dataframes[1])
    data.to_excel('Makro Invoices/Makro Invoices/Saved_Datad.xlsx')
    print("Yes 1")
elif "MM Num. articulo" in dataframes[0] or "MM Num. artículo" in dataframes[0]:
    data = pd.DataFrame(dataframes[0])
    data.to_excel('Makro Invoices/Makro Invoices/Saved_Datad.xlsx')
    print("Yes 2")

elif "Bsdc SL" in dataframes[0] :
    data = pd.DataFrame(dataframes[1])
    data.to_excel('Makro Invoices/Makro Invoices/Saved_Datad.xlsx')
    
elif (len(dataframes)>=3 and ("MM Num. artículo" in dataframes[2] or "MM Num. artículo" in dataframes[2] 
                            or "MM Num. articulo" in dataframes[2])):
    data = pd.DataFrame(dataframes[2])
    data.to_excel('Makro Invoices/Makro Invoices/Saved_Datad.xlsx')
    print('Condition 2')
    
else:
    data = pd.DataFrame(dataframes[0])
    data.to_excel('Makro Invoices/Makro Invoices/Saved_Datad.xlsx')
    print("Yes")

Condition 2


In [83]:
data.head(100)

,MM Num. artículo,Descrip. artículo,Cont,Prec. Ud.,Cont P.,NOT_SELECTED Precio,Cant.,Importe,Imp Prec. Ud. DT,AOO RC
0,Número de pedido,9-89993254,,,,,,,,
1,Entregado a: TASCA 096762,"SANTO DOMINGO, SANTO DOMINGO Granja Flor Valse...",(tasca santo cabra rulo,domingo) unidad 1kg,"24, SAN",CRISTOBAL DE,LA LAGUNA,"38201,",Spain Fecha:,20/05/202
2,,,UD,"10,260",1,"10,26",4,"41,04",0,
3,091867,Tello Codillo asado 700g,PZ,"5,770",1,"5,77",18,"103,86",1,
4,98435223249021,Atún lomo sin piel 2-4piezas/kg,FAO 51,Océano Índico,Oeste,,,,,
5,,,KG,"16,590","4,734","78,54",1,"78,54",1,HD
6,,GTIN: 98435223249021 Lote: 5 GTIN: 98435223249...,LOT: 5,,,,,,,
7,078438,makro Chef Manjares de mar 2x200g,BJ,"2,850",1,"2,85",5,"14,25",1,
8,155329,Altamar Almeja marron congelada,60/80,piezas 1kg,,,,,,
9,,,BL,"3,650",1,"3,65",2,"7,30",0,


In [84]:
# drop cont  Prec. Ud. 
data.columns

Index(['MM Num. artículo', 'Descrip. artículo', 'Cont', 'Prec. Ud.', 'Cont P.',
       'NOT_SELECTED Precio', 'Cant.', 'Importe', 'Imp Prec. Ud. DT',
       'AOO RC'],
      dtype='object')

In [85]:
big = data.columns[0]

In [86]:
cont = data.columns[2]

In [87]:
lot = data.columns[1]

In [88]:
cant = data.columns[-5]

In [89]:
last = data.columns[-1]
sec = data.columns[-2]
third = data.columns[-3]

In [90]:
big

'MM Num. artículo'

In [91]:
da = data.loc[(data[big] != "Número de pedido") & (data[big] != "*** Número de pedido") & (data[big]!="*** Número de") & (data[big]!="*** Número")]

In [92]:
da = da.loc[(da[big] != "Entregado a: TABERNA") & (da[big] !="Entregado a: TASCA") & (da[big]!="Entregado a: TABERNA OSUNA")
          & (da[big]!= "Entregado a: TABERNA osuna")& (da[big]!="Fin de número")]

In [93]:
da = da.loc[da[cont] !="LOT:"]
da = da.loc[da[cant] !="Total"]

In [94]:
print(da.shape)
da.head(100)

(17, 10)


,MM Num. artículo,Descrip. artículo,Cont,Prec. Ud.,Cont P.,NOT_SELECTED Precio,Cant.,Importe,Imp Prec. Ud. DT,AOO RC
1,Entregado a: TASCA 096762,"SANTO DOMINGO, SANTO DOMINGO Granja Flor Valse...",(tasca santo cabra rulo,domingo) unidad 1kg,"24, SAN",CRISTOBAL DE,LA LAGUNA,"38201,",Spain Fecha:,20/05/202
2,,,UD,"10,260",1,"10,26",4,"41,04",0,
3,091867,Tello Codillo asado 700g,PZ,"5,770",1,"5,77",18,"103,86",1,
4,98435223249021,Atún lomo sin piel 2-4piezas/kg,FAO 51,Océano Índico,Oeste,,,,,
5,,,KG,"16,590","4,734","78,54",1,"78,54",1,HD
6,,GTIN: 98435223249021 Lote: 5 GTIN: 98435223249...,LOT: 5,,,,,,,
7,078438,makro Chef Manjares de mar 2x200g,BJ,"2,850",1,"2,85",5,"14,25",1,
8,155329,Altamar Almeja marron congelada,60/80,piezas 1kg,,,,,,
9,,,BL,"3,650",1,"3,65",2,"7,30",0,
10,111089,Signo Arroz bomba bolsa 1 kg,PQ,"2,950",1,"2,95",3,"8,85",1,


In [95]:
db =da.drop([last,sec,third], axis = 1)

In [96]:
db.head(100)

,MM Num. artículo,Descrip. artículo,Cont,Prec. Ud.,Cont P.,NOT_SELECTED Precio,Cant.
1,Entregado a: TASCA 096762,"SANTO DOMINGO, SANTO DOMINGO Granja Flor Valse...",(tasca santo cabra rulo,domingo) unidad 1kg,"24, SAN",CRISTOBAL DE,LA LAGUNA
2,,,UD,"10,260",1,"10,26",4
3,091867,Tello Codillo asado 700g,PZ,"5,770",1,"5,77",18
4,98435223249021,Atún lomo sin piel 2-4piezas/kg,FAO 51,Océano Índico,Oeste,,
5,,,KG,"16,590","4,734","78,54",1
6,,GTIN: 98435223249021 Lote: 5 GTIN: 98435223249...,LOT: 5,,,,
7,078438,makro Chef Manjares de mar 2x200g,BJ,"2,850",1,"2,85",5
8,155329,Altamar Almeja marron congelada,60/80,piezas 1kg,,,
9,,,BL,"3,650",1,"3,65",2
10,111089,Signo Arroz bomba bolsa 1 kg,PQ,"2,950",1,"2,95",3


In [35]:
db = db.applymap(lambda x: str(x.replace(',','.')))

In [49]:
fi = db.astype({'Prec. Ud.':float,'Precio':float,'Importe':float,'Imp':int,
               'Cant.':int})

In [50]:
tot = 0
for i in fi['Importe']:
    tot +=i
print(tot)

539.25


In [51]:
fi.dtypes

MM Num. articulo      object
Descrip. articulo     object
Cont                  object
Prec. Ud.            float64
Cont P.               object
Precio               float64
Cant.                  int32
Importe              float64
Imp                    int32
dtype: object

In [53]:
one_tax = fi.loc[(fi['Imp'] == 1)]

In [61]:
one_tax 
three_tax = 0
for i in one_tax['Importe']:
    three_tax +=i
three_tax = round(three_tax * 0.03,2)

print(three_tax)

1.91


In [55]:
two_tax = fi.loc[(fi['Imp'] == 2)]

In [62]:
two_tax
seven_tax = 0
for i in two_tax['Importe']:
    seven_tax +=i
seven_tax = round(seven_tax *0.07,2)
print(seven_tax)

33.28


In [72]:
final_sum = round(tot+ three_tax + seven_tax,2)
print(final_sum)

574.44


In [160]:
db = da.replace('',np.NaN)

In [161]:

fi = db.dropna(axis=0, thresh=3, how="any") 

In [162]:
fi.to_excel('Makro Invoices/Makro Invoices/Refine.xlsx')
fi.head(100)


,MM Num. artículo,Descrip. artículo,Cont,Prec. Ud.,Cont P.,NOT_SELECTED Precio,Cant.,Importe,Imp Prec.Ud. DT,AOO RC
2,096762,Granja Flor Valsequillo queso,cabra rulo,unidad 1kg,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,UD,"10,260",1,"10,26",4,"41,04",0,NaN
4,091867,Tello Codillo asado 700g,PZ,"5,770",1,"5,77",18,"103,86",1,NaN
5,98435223249021,Atún lomo sin piel 2-4piezas/kg,FAO 51,Océano Índico,Oeste,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,KG,"16,590","4,734","78,54",1,"78,54",1,HD
9,078438,makro Chef Manjares de mar 2x200g,BJ,"2,850",1,"2,85",5,"14,25",1,NaN
10,155329,Altamar Almeja marron congelada,60/80,piezas 1kg,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,BL,"3,650",1,"3,65",2,"7,30",0,NaN
12,111089,Signo Arroz bomba bolsa 1 kg,PQ,"2,950",1,"2,95",3,"8,85",1,NaN
13,451555,makro Chef Mayonesa cubo 51,CB,"17,300",1,"17,30",1,"17,30",1,NaN


In [163]:
fi.columns

Index(['MM Num. artículo', 'Descrip. artículo', 'Cont', 'Prec. Ud.', 'Cont P.',
       'NOT_SELECTED Precio', 'Cant.', 'Importe', 'Imp Prec.Ud. DT', 'AOO RC'],
      dtype='object')

In [164]:
fi['Descrip. artículo'] = fi['Descrip. artículo']+' ' + fi['Cont']+' '+fi['Prec. Ud.'] 

F:\anaconda3\envs\python_3_7\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.



In [176]:
fi.head(100)

,MM Num. artículo,Descrip. artículo,Cont,Prec. Ud.,Cont P.,NOT_SELECTED Precio,Cant.,Importe,Imp Prec.Ud. DT,AOO RC
2,096762,Granja Flor Valsequillo queso cabra rulo unida...,cabra rulo,unidad 1kg,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,UD,"10,260",1,"10,26",4,"41,04",0,NaN
4,091867,"Tello Codillo asado 700g PZ 5,770",PZ,"5,770",1,"5,77",18,"103,86",1,NaN
5,98435223249021,Atún lomo sin piel 2-4piezas/kg FAO 51 Océano ...,FAO 51,Océano Índico,Oeste,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,KG,"16,590","4,734","78,54",1,"78,54",1,HD
9,078438,"makro Chef Manjares de mar 2x200g BJ 2,850",BJ,"2,850",1,"2,85",5,"14,25",1,NaN
10,155329,Altamar Almeja marron congelada 60/80 piezas 1kg,60/80,piezas 1kg,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,BL,"3,650",1,"3,65",2,"7,30",0,NaN
12,111089,"Signo Arroz bomba bolsa 1 kg PQ 2,950",PQ,"2,950",1,"2,95",3,"8,85",1,NaN
13,451555,"makro Chef Mayonesa cubo 51 CB 17,300",CB,"17,300",1,"17,30",1,"17,30",1,NaN


In [166]:
fi.Cont = fi.Cont.shift(-1)

In [94]:
fi.head(100)

,MM Num. artículo,Descrip. artículo,Cont,Prec. Ud.,Cont P.,NOT_SELECTED Precio,Cant.,Importe,Imp Prec.Ud. DT,AOO RC
2,096762,NaN,UD,unidad 1kg,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,PZ,"10,260",1,"10,26",4,"41,04",0,NaN
4,091867,"Tello Codillo asado 700g PZ 5,770 1",FAO 51,"5,770",1,"5,77",18,"103,86",1,NaN
5,98435223249021,Atún lomo sin piel 2-4piezas/kg FAO 51 Océano ...,KG,Océano Índico,Oeste,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,BJ,"16,590","4,734","78,54",1,"78,54",1,HD
9,078438,"makro Chef Manjares de mar 2x200g BJ 2,850 1",60/80,"2,850",1,"2,85",5,"14,25",1,NaN
10,155329,NaN,BL,piezas 1kg,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,PQ,"3,650",1,"3,65",2,"7,30",0,NaN
12,111089,"Signo Arroz bomba bolsa 1 kg PQ 2,950 1",CB,"2,950",1,"2,95",3,"8,85",1,NaN
13,451555,"makro Chef Mayonesa cubo 51 CB 17,300 1",250g,"17,300",1,"17,30",1,"17,30",1,NaN


In [115]:
data = pd.read_excel('Makro Invoices/Makro Invoices/Refine.xlsx')

In [116]:
data.head(100)

,Unnamed: 0,MM Num. artículo,Descrip. artículo,Cont,Prec. Ud.,Cont P.,NOT_SELECTED Precio,Cant.,Importe,Imp Prec.Ud. DT,AOO RC
0,2,9.676200e+04,Granja Flor Valsequillo queso,cabra rulo,unidad 1kg,NaN,NaN,NaN,NaN,NaN,NaN
1,3,NaN,NaN,UD,"10,260",1,"10,26",4.0,"41,04",0.0,NaN
2,4,9.186700e+04,Tello Codillo asado 700g,PZ,"5,770",1,"5,77",18.0,"103,86",1.0,NaN
3,5,9.843522e+13,Atún lomo sin piel 2-4piezas/kg,FAO 51,Océano Índico,Oeste,NaN,NaN,NaN,NaN,NaN
4,6,NaN,NaN,KG,"16,590","4,734","78,54",1.0,"78,54",1.0,HD
5,9,7.843800e+04,makro Chef Manjares de mar 2x200g,BJ,"2,850",1,"2,85",5.0,"14,25",1.0,NaN
6,10,1.553290e+05,Altamar Almeja marron congelada,60/80,piezas 1kg,NaN,NaN,NaN,NaN,NaN,NaN
7,11,NaN,NaN,BL,"3,650",1,"3,65",2.0,"7,30",0.0,NaN
8,12,1.110890e+05,Signo Arroz bomba bolsa 1 kg,PQ,"2,950",1,"2,95",3.0,"8,85",1.0,NaN
9,13,4.515550e+05,makro Chef Mayonesa cubo 51,CB,"17,300",1,"17,30",1.0,"17,30",1.0,NaN


In [137]:
fib.loc[fib['Descrip. artículo'].isnull()==False]

,Unnamed: 0,MM Num. artículo,Descrip. artículo,Cont,Prec. Ud.,Cont P.,NOT_SELECTED Precio,Cant.,Importe,Imp Prec.Ud. DT,AOO RC,Descrip. artículoo
0,2,9.676200e+04,Granja Flor Valsequillo queso cabra rulo unida...,cabra rulo,unidad 1kg,NaN,NaN,NaN,NaN,NaN,NaN,Granja Flor Valsequillo queso cabra rulo unida...
2,4,9.186700e+04,"Tello Codillo asado 700g PZ 5,770",PZ,"5,770",1,"5,77",18.0,"103,86",1.0,NaN,"Tello Codillo asado 700g PZ 5,770"
3,5,9.843522e+13,Atún lomo sin piel 2-4piezas/kg FAO 51 Océano ...,FAO 51,Océano Índico,Oeste,NaN,NaN,NaN,NaN,NaN,Atún lomo sin piel 2-4piezas/kg FAO 51 Océano ...
5,9,7.843800e+04,"makro Chef Manjares de mar 2x200g BJ 2,850",BJ,"2,850",1,"2,85",5.0,"14,25",1.0,NaN,"makro Chef Manjares de mar 2x200g BJ 2,850"
6,10,1.553290e+05,Altamar Almeja marron congelada 60/80 piezas 1kg,60/80,piezas 1kg,NaN,NaN,NaN,NaN,NaN,NaN,Altamar Almeja marron congelada 60/80 piezas 1kg
8,12,1.110890e+05,"Signo Arroz bomba bolsa 1 kg PQ 2,950",PQ,"2,950",1,"2,95",3.0,"8,85",1.0,NaN,"Signo Arroz bomba bolsa 1 kg PQ 2,950"
9,13,4.515550e+05,"makro Chef Mayonesa cubo 51 CB 17,300",CB,"17,300",1,"17,30",1.0,"17,30",1.0,NaN,"makro Chef Mayonesa cubo 51 CB 17,300"
12,16,1.562680e+05,"Gp Leche condensada botella 1kg BT 2,320",BT,"2,320",1,"2,32",4.0,"9,28",0.0,NaN,"Gp Leche condensada botella 1kg BT 2,320"
13,17,8.039200e+05,Castillo de chiva Vino blanco moscatel 75cl,moscatel,75cl,NaN,NaN,NaN,NaN,NaN,NaN,Castillo de chiva Vino blanco moscatel 75cl
15,19,7.244700e+04,"Clipper Recarga soplete UD 5,250",UD,"5,250",1,"5,25",4.0,"21,00",2.0,NaN,"Clipper Recarga soplete UD 5,250"


In [134]:
fib = data 
#df[(df['SystemA'].notnull()) & (df['SystemB'].notnull())]
fib[(fib['Descrip. artículo'].notnull()) & (fib['Cont'].notnull())]
#    fib['Descrip. artículo'] = fib['Descrip. artículo']+' ' + fib['Cont']+' '+fib['Prec. Ud.'] +' '+ fib['Cont P.']
#else:
 #   fib['Descrip. artículo'] = fib['Descrip. artículo']+' ' + fib['Cont']+' '+fib['Prec. Ud.'] 

,Unnamed: 0,MM Num. artículo,Descrip. artículo,Cont,Prec. Ud.,Cont P.,NOT_SELECTED Precio,Cant.,Importe,Imp Prec.Ud. DT,AOO RC,Descrip. artículoo
0,2,9.676200e+04,Granja Flor Valsequillo queso cabra rulo unida...,cabra rulo,unidad 1kg,NaN,NaN,NaN,NaN,NaN,NaN,Granja Flor Valsequillo queso cabra rulo unida...
2,4,9.186700e+04,"Tello Codillo asado 700g PZ 5,770",PZ,"5,770",1,"5,77",18.0,"103,86",1.0,NaN,"Tello Codillo asado 700g PZ 5,770"
3,5,9.843522e+13,Atún lomo sin piel 2-4piezas/kg FAO 51 Océano ...,FAO 51,Océano Índico,Oeste,NaN,NaN,NaN,NaN,NaN,Atún lomo sin piel 2-4piezas/kg FAO 51 Océano ...
5,9,7.843800e+04,"makro Chef Manjares de mar 2x200g BJ 2,850",BJ,"2,850",1,"2,85",5.0,"14,25",1.0,NaN,"makro Chef Manjares de mar 2x200g BJ 2,850"
6,10,1.553290e+05,Altamar Almeja marron congelada 60/80 piezas 1kg,60/80,piezas 1kg,NaN,NaN,NaN,NaN,NaN,NaN,Altamar Almeja marron congelada 60/80 piezas 1kg
8,12,1.110890e+05,"Signo Arroz bomba bolsa 1 kg PQ 2,950",PQ,"2,950",1,"2,95",3.0,"8,85",1.0,NaN,"Signo Arroz bomba bolsa 1 kg PQ 2,950"
9,13,4.515550e+05,"makro Chef Mayonesa cubo 51 CB 17,300",CB,"17,300",1,"17,30",1.0,"17,30",1.0,NaN,"makro Chef Mayonesa cubo 51 CB 17,300"
12,16,1.562680e+05,"Gp Leche condensada botella 1kg BT 2,320",BT,"2,320",1,"2,32",4.0,"9,28",0.0,NaN,"Gp Leche condensada botella 1kg BT 2,320"
13,17,8.039200e+05,Castillo de chiva Vino blanco moscatel 75cl,moscatel,75cl,NaN,NaN,NaN,NaN,NaN,NaN,Castillo de chiva Vino blanco moscatel 75cl
15,19,7.244700e+04,"Clipper Recarga soplete UD 5,250",UD,"5,250",1,"5,25",4.0,"21,00",2.0,NaN,"Clipper Recarga soplete UD 5,250"


In [124]:
fib.head(100)

,Unnamed: 0,MM Num. artículo,Descrip. artículo,Cont,Prec. Ud.,Cont P.,NOT_SELECTED Precio,Cant.,Importe,Imp Prec.Ud. DT,AOO RC,Descrip. artículoo
0,2,9.676200e+04,Granja Flor Valsequillo queso cabra rulo unida...,cabra rulo,unidad 1kg,NaN,NaN,NaN,NaN,NaN,NaN,Granja Flor Valsequillo queso cabra rulo unida...
1,3,NaN,NaN,UD,"10,260",1,"10,26",4.0,"41,04",0.0,NaN,NaN
2,4,9.186700e+04,"Tello Codillo asado 700g PZ 5,770",PZ,"5,770",1,"5,77",18.0,"103,86",1.0,NaN,"Tello Codillo asado 700g PZ 5,770"
3,5,9.843522e+13,Atún lomo sin piel 2-4piezas/kg FAO 51 Océano ...,FAO 51,Océano Índico,Oeste,NaN,NaN,NaN,NaN,NaN,Atún lomo sin piel 2-4piezas/kg FAO 51 Océano ...
4,6,NaN,NaN,KG,"16,590","4,734","78,54",1.0,"78,54",1.0,HD,NaN
5,9,7.843800e+04,"makro Chef Manjares de mar 2x200g BJ 2,850",BJ,"2,850",1,"2,85",5.0,"14,25",1.0,NaN,"makro Chef Manjares de mar 2x200g BJ 2,850"
6,10,1.553290e+05,Altamar Almeja marron congelada 60/80 piezas 1kg,60/80,piezas 1kg,NaN,NaN,NaN,NaN,NaN,NaN,Altamar Almeja marron congelada 60/80 piezas 1kg
7,11,NaN,NaN,BL,"3,650",1,"3,65",2.0,"7,30",0.0,NaN,NaN
8,12,1.110890e+05,"Signo Arroz bomba bolsa 1 kg PQ 2,950",PQ,"2,950",1,"2,95",3.0,"8,85",1.0,NaN,"Signo Arroz bomba bolsa 1 kg PQ 2,950"
9,13,4.515550e+05,"makro Chef Mayonesa cubo 51 CB 17,300",CB,"17,300",1,"17,30",1.0,"17,30",1.0,NaN,"makro Chef Mayonesa cubo 51 CB 17,300"


In [125]:
def fun(df):
    if df.columns[0]== NULL  and df.columns[1] == NULL:
        df[-9] = df[-9].shift(-1)
        df[-8] = df[-8].shift(-1)
        df[-7] = df[-7].shift(-1)
        df[-6] = df[-6].shift(-1)
        df[-5] = df[-5].shift(-1)
        df[-4] = df[-4].shift(-1)
        df[-3] = df[-3].shift(-1)
        df[-2] = df[-2].shift(-1)
        df[-1] = df[-1].shift(-1)
    

In [127]:
df = fib.map(fun)

AttributeError: 'DataFrame' object has no attribute 'map'

In [177]:
l = 0
for r in range(0,100):
    for c in range(1):
        if fi.iloc[r,c] == NaN:
            k=k+1

        else:
            print("Nan Detected")
            l=l+1
print(l," Nan Values encountered")

NameError: name 'NaN' is not defined

In [183]:
import pandas as pd
import numpy as np

dfa = pd.DataFrame(
	[[np.nan, 72, 67],
	[23, 78, 62],
	[32, 74, np.nan],
	[np.nan, 54, 76]])

for i in range(df.shape[0]): #iterate over rows
    for j in range(df.shape[1]): #iterate over columns
        value = df.at[i, j] #get cell value
        #print(np.isnan(value), end="\t")
        if (np.isnan(value)):
            print('HI')
    print()

HI


HI

HI



In [182]:
for i in range(df.shape[0]):
    if (np.isnan(i)):
        print('hi')

In [184]:
dfa.head()

,0,1,2
0,NaN,72,67.0
1,23.0,78,62.0
2,32.0,74,NaN
3,NaN,54,76.0


In [217]:
import re
for i in fi[big]:
    if not  re.findall("[0-9]", str(i)):
        fi.Cont = fi.Cont.shift(-1)
        fi['Prec. Ud.'] = fi['Prec. Ud.'].shift(-1)
        fi['Cont P.'] = fi['Cont P.'].shift(-1)
        fi['NOT_SELECTED Precio'] = fi['NOT_SELECTED Precio'].shift(-1)
        fi['Cant.'] = fi['Cant.'].shift(-1)
        fi['Importe'] = fi['Importe'].shift(-1)
        fi['Imp Prec.Ud. DT'] =fi['Imp Prec.Ud. DT'].shift(-1)
        fi['AOO RC'] = fi['AOO RC'].shift(-1)
        
        print('NAN',i)

NAN nan
NAN nan
NAN nan
NAN nan
NAN nan


In [218]:
fi.head(100)

,MM Num. artículo,Descrip. artículo,Cont,Prec. Ud.,Cont P.,NOT_SELECTED Precio,Cant.,Importe,Imp Prec.Ud. DT,AOO RC
2,9.676200e+04,Granja Flor Valsequillo queso cabra rulo unida...,UD,"2,850",1,"2,85",5,"14,25",1,NaN
3,NaN,NaN,NaN,piezas 1kg,NaN,NaN,NaN,NaN,NaN,NaN
4,9.186700e+04,"Tello Codillo asado 700g PZ 5,770",NaN,"3,650",1,"3,65",2,"7,30",0,NaN
5,9.843522e+13,Atún lomo sin piel 2-4piezas/kg FAO 51 Océano ...,NaN,"2,950",1,"2,95",3,"8,85",1,NaN
6,NaN,NaN,NaN,"17,300",1,"17,30",1,"17,30",1,NaN
9,7.843800e+04,"makro Chef Manjares de mar 2x200g BJ 2,850",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,1.553290e+05,Altamar Almeja marron congelada 60/80 piezas 1kg,NaN,"2,160",1,"2,16",4,"8,64",1,NaN
11,NaN,NaN,NaN,"2,320",1,"2,32",4,"9,28",0,NaN
12,1.110890e+05,"Signo Arroz bomba bolsa 1 kg PQ 2,950",NaN,75cl,NaN,NaN,NaN,NaN,NaN,NaN
13,4.515550e+05,"makro Chef Mayonesa cubo 51 CB 17,300",NaN,"3,010",1,"3,01",5,"15,05",2,NaN


In [202]:
fi.dtypes

MM Num. artículo       float64
Descrip. artículo       object
Cont                    object
Prec. Ud.               object
Cont P.                 object
NOT_SELECTED Precio     object
Cant.                   object
Importe                 object
Imp Prec.Ud. DT         object
AOO RC                  object
dtype: object

In [196]:
fi = fi.astype({big:float})